In [2]:
# Re-import simulation and thermodynamic classes and utilities from previous code

import numpy as np
import pandas as pd
from collections import deque
from matplotlib import pyplot as plt
from matplotlib.patches import Polygon

# Define Bird and Model classes
class Bird:
    def __init__(self, pos, vel, speed, cohere_factor, separation,
                 separate_factor, match_factor, visual_distance, ensemble, history_length):
        self.pos = np.array(pos, dtype=np.float64)
        self.vel = np.array(vel, dtype=np.float64)
        norm_vel = np.linalg.norm(self.vel)
        self.vel = self.vel / norm_vel if norm_vel > 0 else self.vel
        self.speed = speed
        self.cohere_factor = cohere_factor
        self.separation = separation
        self.separate_factor = separate_factor
        self.match_factor = match_factor
        self.visual_distance = visual_distance
        self.ensemble = ensemble
        self.ensemble_history = deque(maxlen=history_length)

class Model:
    def __init__(self, n_birds=100, speed=2.0, cohere_factor=0.4, separation=4.0,
                 separate_factor=0.25, match_factor=0.02, visual_distance=5.0,
                 ensemble=1, history_length=8, extent=(50, 50), seed=1234):
        self.iteration = 0
        self.extent = extent
        self.agents = []
        self.rng = np.random.default_rng(seed)
        for _ in range(n_birds):
            vel = self.rng.uniform(-1, 1, size=2)
            vel = vel / np.linalg.norm(vel) if np.linalg.norm(vel) > 0 else np.array([1.0, 0.0])
            pos = self.rng.uniform(low=[0, 0], high=extent, size=2)
            bird = Bird(pos, vel, speed, cohere_factor, separation,
                        separate_factor, match_factor, visual_distance,
                        ensemble, history_length)
            self.agents.append(bird)

def move_agent(agent, model):
    agent.pos += agent.vel * agent.speed
    agent.pos[0] %= model.extent[0]
    agent.pos[1] %= model.extent[1]

def get_neighbors(agent, model, radius):
    return [other for other in model.agents if other is not agent and np.linalg.norm(agent.pos - other.pos) < radius]

def agent_step(agent, model):
    neighbors = get_neighbors(agent, model, agent.visual_distance)
    match = separate = cohere = np.zeros(2)
    angle_threshold = np.deg2rad(120)
    bird_speed = np.linalg.norm(agent.vel)

    for neighbor in neighbors:
        heading = neighbor.pos - agent.pos
        cohere += heading
        if np.linalg.norm(heading) < agent.separation:
            separate -= heading
        match += neighbor.vel
    N = max(len(neighbors), 1)
    agent.ensemble_history.append(len(neighbors))
    median_history = np.median(agent.ensemble_history)
    agent.ensemble = 1 / (1 + median_history)

    cohere = (cohere / N) * agent.cohere_factor
    separate = (separate / N) * agent.separate_factor
    match = (match / N) * agent.match_factor
    new_vel = (agent.vel + cohere + separate + match) / 2.0
    norm_new_vel = np.linalg.norm(new_vel)
    agent.vel = new_vel / norm_new_vel if norm_new_vel > 0 else agent.vel
    move_agent(agent, model)

def model_step(model): model.iteration += 1
def Step_Num(model): return model.iteration
def Model_Ensemble_Energy(model): return sum(b.ensemble for b in model.agents)

def Model_Energy_DistributionFN(model, bins=10):
    energies = np.array([b.ensemble for b in model.agents])
    counts, bin_edges = np.histogram(energies, bins=bins)
    total = np.sum(counts)
    pdf = counts / total if total > 0 else counts
    midpoints = (bin_edges[:-1] + bin_edges[1:]) / 2
    return list(zip(midpoints, pdf))

def Model_Entropy(e_p_array):
    return -sum(p * np.log(p) for e, p in e_p_array if p > 0)

def Model_Temperature(e_p_array):
    result = 0.0
    for i in range(1, len(e_p_array)):
        E_prev, p_prev = e_p_array[i - 1]
        E, p = e_p_array[i]
        delta_E = E - E_prev
        if delta_E == 0: continue
        dp_dE = (p - p_prev) / delta_E
        if not (1e-8 < abs(dp_dE) < 1e8): continue
        result += dp_dE * np.log(p) if p > 0 else 0
    return abs(-1 / result) if result != 0 else 0

def Model_Heat(S, S_prev, T): return abs(T) * (S - S_prev)
S_prev_global = None
def Model_Thermo_State(model):
    global S_prev_global
    e_p_array = Model_Energy_DistributionFN(model)
    S = Model_Entropy(e_p_array)
    T = Model_Temperature(e_p_array)
    dQ = Model_Heat(S, S_prev_global, T) if S_prev_global is not None else 0.0
    S_prev_global = S
    return {"S": S, "T": T, "dQ": dQ, "e_p_array": e_p_array}

def Model_Internal_Energy(model):
    return sum(E * p for E, p in Model_Energy_DistributionFN(model))

def Model_Volume(model):
    total = count = 0
    for i, a in enumerate(model.agents[:-1]):
        for b in model.agents[i+1:]:
            total += np.linalg.norm(a.pos - b.pos)
            count += 1
    avg = total / count if count > 0 else 0
    return 1 / avg if avg > 0 else 0

def run_simulation(model, step_num, agent_data_funcs, model_data_funcs):
    agent_records, model_records = [], []
    for _ in range(step_num):
        for agent in model.agents: agent_step(agent, model)
        model_step(model)
        model_record = {func.__name__: func(model) for func in model_data_funcs}
        model_records.append(model_record)
    return agent_records, model_records

def merge_e_p_array(ep_tmp, e_p_sums):
    new_e_p_array = []
    for i in range(len(ep_tmp)):
        merged = e_p_sums[i].copy()
        for E, p in ep_tmp[i]:
            for j, (E_sum, p_sum) in enumerate(merged):
                if np.isclose(E_sum, E):
                    merged[j] = (E_sum, p_sum + p)
                    break
            else:
                merged.append((E, p))
        new_e_p_array.append(merged)
    return new_e_p_array

def normalize_e_p_array(e_p_array):
    normalized = []
    for dist in e_p_array:
        total = sum(p for _, p in dist)
        normed = [(e, p / total) for e, p in dist] if total > 0 else dist
        normalized.append(normed)
    return normalized

# Define agent/model data funcs
agent_data_funcs = ["separation", "speed", "ensemble"]
model_data_funcs = [Step_Num, Model_Ensemble_Energy, Model_Thermo_State, Model_Volume, Model_Internal_Energy]


In [3]:
# Run simplified aggregation over 30 simulations
num_simulations = 30
step_num = 150

# First simulation to initialize
model = Model(seed=1)
_, model_data = run_simulation(model, step_num, agent_data_funcs, model_data_funcs)
model_df = pd.DataFrame(model_data)

energy_sums = model_df["Model_Ensemble_Energy"].to_numpy().copy()
S_sums = model_df["Model_Thermo_State"].apply(lambda x: x["S"]).to_numpy().copy()
T_sums = model_df["Model_Thermo_State"].apply(lambda x: x["T"]).to_numpy().copy()
dQ_sums = model_df["Model_Thermo_State"].apply(lambda x: x["dQ"]).to_numpy().copy()
V_sums = model_df["Model_Volume"].to_numpy().copy()
U_sums = model_df["Model_Internal_Energy"].to_numpy().copy()
e_p_sums = model_df["Model_Thermo_State"].apply(lambda x: x["e_p_array"]).tolist()

# Repeat for remaining simulations
for i in range(2, num_simulations + 1):
    model = Model(seed=i)
    _, model_data_temp = run_simulation(model, step_num, agent_data_funcs, model_data_funcs)
    model_df_temp = pd.DataFrame(model_data_temp)
    
    energy_sums += model_df_temp["Model_Ensemble_Energy"].to_numpy()
    S_sums += model_df_temp["Model_Thermo_State"].apply(lambda x: x["S"]).to_numpy()
    T_sums += model_df_temp["Model_Thermo_State"].apply(lambda x: x["T"]).to_numpy()
    dQ_sums += model_df_temp["Model_Thermo_State"].apply(lambda x: x["dQ"]).to_numpy()
    V_sums += model_df_temp["Model_Volume"].to_numpy()
    U_sums += model_df_temp["Model_Internal_Energy"].to_numpy()
    
    ep_tmp = model_df_temp["Model_Thermo_State"].apply(lambda x: x["e_p_array"]).tolist()
    e_p_sums = merge_e_p_array(ep_tmp, e_p_sums)

# Compute final averages
average_energy = energy_sums / num_simulations
average_S = S_sums / num_simulations
average_T = T_sums / num_simulations
average_dQ = dQ_sums / num_simulations
average_V = V_sums / num_simulations
average_U = U_sums / num_simulations
average_e_p_array = normalize_e_p_array(e_p_sums)


KeyboardInterrupt: 